In [1]:
# Libreria tensor flowe
import tensorflow as tf
# Daset de datos con números escritos a manorom 
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow_datasets as tfds
# API de tensorfow para construir y entrenar modelos de alto nivel
from tensorflow import keras
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [18]:
# Se recuperan los datos y el metadata de los datos
#mnist_train2, mnist_info = tfds.load('mnist', split='train', as_supervised=True,  shuffle_files=True,with_info=True)
# De los datos, se extrae los datos de entrenamiento y test
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 784)
x_test = x_test.reshape(x_test.shape[0], 784)

# El formato original esta en "unit8" con lo que se tiene que convertir en un float32 para que funcione
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#Se divide la imagen en 255 porque es la escala de grises y es mejor operar con 0 y 1. Más sencillo
x_train, x_test = x_train / 255.0, x_test / 255.0

# Paso necesario porque devuelve los registros y toda la información en la misma columna. Pero 
# se necesita esos valores esten en un matriz de 10 columnas para calcular bien los acietros.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

mnist_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))



In [3]:
# El ejemplo de .map es lo mismo que se ha hecho arriba cuando se han leido todos los datos
# El TFDS devuelve las imagenes en uint8, mientras que la red espera float 32
# hay que normalizar para que funcione
#def normalize_img(image, label):
#  """Normalizes images: `uint8` -> `float32`."""
#  return tf.cast(image, tf.float32) / 255, label
#
# map permite realizar una transformación a cada uno de los datos del conjuntos de datos
# que lo compone. En este caso lo que se se hace convertir las imagenes del TDFS al formato
# compatible de la red.
#mnist_train = mnist_train.map(
#    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
# Activa que los datos se guarden cache para mejorar el rendimiento
mnist_train = mnist_train.cache()
# Activa que los datos se recuperan de manera aleatoria
#mnist_train = mnist_train.shuffle(mnist_info.splits['train'].num_examples)
# Número de lotes que hará en cada iteracción
mnist_train = mnist_train.batch(100)


In [4]:
# Imagen del número descompuesto a un vector. Se informa que el vector es de 784 porque es
# multiplicación de 28 x 28 que son los pixeles de la imagen
# El valor None es para indicar que el valor, o filas, son variables según el número de imagenes
x = tf.keras.Input(name='x', shape=(None,784), dtype=tf.float32) 
# Matriz de pesos, que se inicializa a 0, con el tf.zeros. 
# La matriz tiene 784 filas, que son el número de pixeles. Y 10 columnas porque
# la siguienet capa tiene 10 elementos. La siguiente capa es la de salida
P = tf.Variable(tf.zeros([784,10]))
# Son los bias, o sesgo. Será un vector de 10 elementos inicializado a 0.
b = tf.Variable(tf.zeros([10]))
# Es la operación de salida. Aquí es la formula que se explica en la parte de redes
# neuronales, como funciona. 
# la formula es la multiplicación de la matriz de entrada * peso, más el sesgo.
y = tf.matmul(x,P)+b
# Matriz con las etiquetas reales del set de datos. Que tendrá un nuevo variable de filas
# y 10 columnas.
yR = tf.keras.Input(name='yR', shape=(None,10), dtype=tf.float32) 

In [5]:
print("x ", x.shape)
print("P ", P.shape)
print("b ", b.shape)

print("yr ", yR.shape)
print("y ", y.shape)

x  (None, None, 784)
P  (784, 10)
b  (10,)
yr  (None, None, 10)
y  (None, None, 10)


In [6]:
def compute_loss():
    log_x = tf.math.log(x)
    m = tf.math.square(log_x)
    return m

In [15]:
# Funcion para ver como esta aprendiendo el algoritmo
def avance(epoca_i,X_data, Y_data):
    x = X_data
    yR = Y_data
          
    y = tf.matmul(X_data,P)+b    

    #print("x ", x.shape)
    #print("P ", P.shape)
    #print("b ", b.shape)

    #print("yr ", Y_data.shape)
    #print("y ", y.shape)

    # Se definen los algoritmos de como va aprender la red en cada una de las iteracciones
    # Softmax nos va dar un vector de 10 elementos para generar la predicción pero en probabilidades
    # Esto nos va a dar un vector donde en cada posición nos va dar el % de probalidades que sea.
    # Es la libreria es la que nos va dar el error de nuestra predicciones. Se le pasará las 
    # etiquetas de los números y la salida de la red para que nos diga que tan bien o tan mal esta
    # el algoritmo para repartir los pesos.
    softmax = tf.nn.softmax_cross_entropy_with_logits(labels=Y_data,logits=y)
    
    # Es la función del coste de la predicción este número tiene que tender a 0.
    costo = tf.reduce_mean(softmax)
    
    # Código original tf.train.GradientDescentOptimizer(0.5).minimize(costo)
    # Es el que permitir ajustar los pesos de nuestra matriz. Es decir, si el costo = 2, ajustará los pesos
    # para que ese costo tienda a 0.
    optimizador = tf.keras.optimizers.SGD(learning_rate=0.5,momentum=0.0, nesterov=False) #.minimize(compute_loss,costo)
    print(optimizador)

    # Da el arreglo de booleanos para decirnos cuales estan bien y cuales están mal
    # argmax nos va dar el valor más alto de todas las predicciones que hizo softmax. 
    # Y se va comparar con los datos reales para decirnos si se hizo bien o mal la
    # predicción
    #prediccion = tf.equal(tf.argmax(y,1),tf.argmax(yR,1))

    # Da el % sobre el arreglo  de predicción. Si en la variable de predicción hay
    # 10 elementos, 5 están bien y 5 están mal. El % será edl 50%
    #accuracy = tf.reduce_mean(tf.cast(prediccion, tf.float32))

    #if(epoca_i%50==0):
    #    print('Epoca: {:<4} - Costo: {:<8.3} Certeza: {:<5.3}'.format(epoca_i,costo,accuracy))


In [16]:
for epoch in range(1):       
    ds = mnist_train.take(10)
    for image, label in tfds.as_numpy(ds):        
        avance(epoch, image, label)            